In [1]:
import os
os.chdir('/Users/ericchuu/fit-af/src')
from dotenv import load_dotenv
from athlete.Athlete import Athlete
# Load credentials from .env file
load_dotenv()
email = os.getenv('GARMIN_EMAIL')
password = os.getenv('GARMIN_PASSWORD')
intervals_icu_id = os.getenv('INTERVALS_ICU_ID')
training_plan_dir = "/Users/ericchuu/training_plan"

import json

user_prompt = '''
{
  "sex": "male",
  "age": 28,
  "goal_event": {
    "event": 5000,
    "goal_time": {
      "hours": 0,
      "minutes": 18,
      "seconds": 0
    }
  },
  "timeline_weeks": 2,
  "recent_time_trial": {
    "event": 5000,
    "hours": 0,
    "minutes": 20,
    "seconds": 0
  },
  "training_days_per_week": 4,
  "start_date": "2025-02-10"
}
'''
user_data = json.loads(user_prompt)

athlete = Athlete(
    user_id=1,
    email=email,
    password=password,
    intervals_icu_id=intervals_icu_id,
    training_plan_input=user_data,
    training_plan_dir=training_plan_dir
  )


/Users/ericchuu/fit-af/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# fetch activities from last 30 days and write to activities db
athlete.fetch_historical_activities()

In [3]:
# Look at the activities written to the activities table
import psycopg2
import pandas as pd

db_params = {
    "dbname": os.getenv('DB_NAME'),
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "host": os.getenv('DB_HOST'),
    "port": os.getenv('DB_PORT')
}

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

query = "SELECT activity_id, activity_type, duration, distance FROM activities LIMIT 10;"
cursor.execute(query)

rows = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]

activities_df = pd.DataFrame(rows, columns=colnames)
print(activities_df)

   activity_id   activity_type      duration       distance
0  18254316610    lap_swimming   3758.693115    2880.360107
1  18227215988   trail_running  44328.996094  100484.062500
2  18218061573         running   1937.411987    5762.129883
3  18208135270  indoor_cycling   4502.692871   37764.968750
4  18201251695         running   5102.536133   14446.169922
5  18191876024  indoor_cycling   4520.369141   38725.921875
6  18173495244   trail_running   7525.059082   21143.740234
7  18164335317  indoor_cycling   2826.054932   23601.890625
8  18155081174   trail_running   7835.653809   21259.990234
9  18147054258         running   2702.093994    8125.640137
